In [0]:
try: 
  dbutils.notebook.run(path = "/Users/siddardha.kaja@aircanada.ca/DP-ETL Framework/prod/Generic-Mount", arguments={'project_name': 'pm_ap_conn'}, timeout_seconds=28000)
  print("pm_ap_conn is mounted")
except:
  print("pm_ap_conn is already mounted")

pm_ap_conn is already mounted


In [0]:
pip install sentencepiece

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
pip install contractions==0.1.73

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
pip install protobuf==5.28.3

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
startDate = '2024-10-20'
endDate = '2024-10-25'

In [0]:
def cleaningForSentimentAnalysis(startDate, endDate):
  from transformers import pipeline
  from pyspark.sql.types import StringType, MapType, FloatType
  from pyspark.sql.functions import udf, col, lit, create_map, lower
  from pyspark import StorageLevel
  spark.conf.set("spark.sql.shuffle.partitions", "31")

 

  def spellingCorrectionModel():
    modelName = "oliverguhr/spelling-correction-english-base"
    model = pipeline("text2text-generation",model= modelName, tokenizer = modelName)
    return model

  def commentSpellingCorrection(model, text):
    correctedSpelling = model(text, max_length = len(text)+1)
    return correctedSpelling[0]

  def spellingCorrection(pysparkDf, colName):
    spellingCorrectionModelFunc = spellingCorrectionModel()
    feed = pysparkDf.repartition(31)
    spellingCorrectionSchema = MapType(StringType(), StringType())
    spellingCorrectionUdf = udf(lambda comments: commentSpellingCorrection(spellingCorrectionModelFunc, comments), spellingCorrectionSchema)
    result = feed.withColumn("correctedSpelling", spellingCorrectionUdf(col(colName))).withColumn("correctedSpellingText", col("correctedSpelling").getItem("generated_text"))
    return result

  def detectLanguageModel():
    modelName = "papluca/xlm-roberta-base-language-detection"
    translate = pipeline("text-classification", model=modelName, tokenizer = modelName, max_length=512, truncation=True)
    return translate

  def commentLanguageDetection(model, text):
    commentLanguage = model(text)
    return commentLanguage[0]

  def languageDetection(pysparkDf, colName):
    languageModel = detectLanguageModel()
    feed = pysparkDf.repartition(31)
    langSchema = MapType(StringType(), StringType())
    langSparkUdf = udf(lambda comments: commentLanguageDetection(languageModel, comments), langSchema)
    result = feed.withColumn("langDetection", langSparkUdf(col(colName))).withColumn("detectedLang", col("langDetection").getItem("label")).withColumn("detectedLangProbability", col("langDetection").getItem("score"))
    return result

  def preprocess(df):
    """
    """
    from pyspark.sql import functions as F
    from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField
    # remove the records whose comments are null
    dfRemoveNull = df.filter(df.comment != "")
    preprocess_udf = F.udf(lambda text: " ".join(['@user' if t.startswith('@') and len(t) > 1 else '@user' if t.startswith('​@') and len(t) > 1 else 'http' if t.startswith('http') else t for t in text.split(" ")]), StringType())
    processedDf = dfRemoveNull.withColumn("processedComment", preprocess_udf(dfRemoveNull["comment"]))
    return processedDf
    
  def cleanText(pysparkDf, slangDict):
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import udf, col
    from pyspark.sql.types import StringType
    import contractions

    def preCleanText(inputText, reverse):
      if reverse == False:
        inputText1 = inputText.replace(',', ' , ')
        inputText2 = inputText1.replace('!', ' ! ')
        inputText3 = inputText2.replace('.', ' . ')
      elif reverse == True:
        inputText1 = inputText.replace(' , ', ',')
        inputText2 = inputText1.replace(' ! ', '!')
        inputText3 = inputText2.replace(' . ', '.')
      return inputText3
    def expandText(text, slangDict):
      inputText = preCleanText(text, reverse = False)
      words = inputText.split()
      expandedText = ' '.join(slangDict.get(word.lower(), word).upper() if word.isupper() else slangDict.get(word.lower(), word) for word in words)
      outputText = preCleanText(expandedText, reverse = True)
      #apply contractions
      fixText = str(contractions.fix(outputText))
      return fixText

    
    def main(pysparkDf, slangDict):
      expandTextUDF = udf(lambda text: expandText(text, slangDict), StringType())

      result = pysparkDf.withColumn("expandedText", expandTextUDF(col("processedComment")))
      # result.write.format("delta").mode("overwrite").option("overwriteSchema", "True").save(sentimentsMountPath)
      return result
    return main(pysparkDf, slangDict)
  
  def main():
    import ast
    with open('/Workspace/Users/siddardha.kaja@aircanada.ca/EnglishSDK/slangDict.txt') as f:
      slang_dict_str = f.read()

    slangDict = ast.literal_eval(slang_dict_str)

    youtubeVideoAttributesMain = spark.read.format("delta").load("/mnt/Sandbox/pm_ap_conn/sid_test/youtube/youtubeVideoAttributes")

    youtubeVideoCommentsReplies = spark.read.format("delta").load("/mnt/Sandbox/pm_ap_conn/sid_test/youtube/youtubeVideoCommentsReplies")

    sampleSize = 0.001
    commentDateRange = ('2024-10-01', '2024-10-25')
    youtubeVideoCommentsRepliesFiltered = youtubeVideoCommentsReplies.join(youtubeVideoAttributesMain,youtubeVideoCommentsReplies.videoId ==  youtubeVideoAttributesMain.videoId,"inner").select( youtubeVideoAttributesMain["title"], youtubeVideoAttributesMain["description"], youtubeVideoAttributesMain["tags"], youtubeVideoCommentsReplies["videoId"], youtubeVideoCommentsReplies["commentId"], youtubeVideoCommentsReplies["commentUpdatedDate"], youtubeVideoCommentsReplies["commentAuthorChannelId"], youtubeVideoCommentsReplies["comment"]).filter((youtubeVideoAttributesMain.activeFlag == "Y") & (youtubeVideoCommentsReplies.commentUpdatedDate.between(*commentDateRange)) ).sample(sampleSize)


    # <____________________________>
    youtubeVideoCommentsRepliesFiltered = youtubeVideoCommentsRepliesFiltered.repartition(31)
    # preprocess to remove the usernames and urls from the comments
    preprocessedDf = preprocess(youtubeVideoCommentsRepliesFiltered)

    #pass preprocessed df to get the language detected
    langDetected = languageDetection(preprocessedDf, "comment")
    langDetected.cache()
    # #filter en comments and filter them into a new df
    langFilteredEN = langDetected.filter(langDetected.detectedLang == "en")

    #clean the text by expanding slags and contractions
    cleanedTextDf = cleanText(langFilteredEN, slangDict)

    # # correct spellings
    # correctedSpellingsDf = spellingCorrection(cleanedTextDf, "expandedText")

    # scoop the records that are not en into another df
    langFilteredNonEN = langDetected.filter(langDetected.detectedLang != "en")
    # create cols to match the en df for further steps
    # langFilteredNonEN_1 = langFilteredNonEN.withColumn('expandedText', lit("None")).withColumn('correctedSpelling', create_map(lit("None"), lit("None"))).withColumn('correctedSpellingText', col("comment"))
    langFilteredNonEN_1 = langFilteredNonEN.withColumn('expandedText', lit("comment"))
    # # combine the en and non-en dfs 
    forSentiAnalysis = cleanedTextDf.union(langFilteredNonEN_1)
      
    # #do sentiment analysis
    # finalDf = commentSentimentAnalysis(forSentiAnalysis, "expandedText")
    return(youtubeVideoCommentsRepliesFiltered)
 
  return main()

In [0]:
preprocessed = cleaningForSentimentAnalysis(startDate, endDate)
display(preprocessed)

title,description,tags,videoId,commentId,commentUpdatedDate,commentAuthorChannelId,comment
